# Load data and go to directory with python scripts

In [2]:
scripts_path = '/Users/jendawk/Dropbox (MIT)/C Diff Recurrence Paper/Analyses/scripts/'

# Path to save figures
path_to_save = '/'.join(scripts_path.split('/')[:-2]) + '/'

# Path to save supplementary and figure data folders
save_path = '/Users/jendawk/Dropbox (MIT)/C Diff Recurrence Paper/'
%cd $scripts_path

/Users/jendawk/Dropbox (MIT)/C Diff Recurrence Paper/Analyses/scripts


In [1]:
from dataLoader import *
from helper import *
import scipy.stats as st
from collections import Counter
import sklearn
from matplotlib import cm
import scipy
import  itertools
from datetime import datetime

from seaborn import clustermap
from scipy.cluster.hierarchy import linkage
from sklearn.linear_model import LogisticRegression
import os
import time
import pickle as pkl
import re
from matplotlib_venn import venn3, venn2
from statsmodels.stats.multitest import multipletests
dl = dataLoader(pt_perc = {'metabs': .25, '16s': .05, 'scfa': 0, 'toxin':0}, meas_thresh = 
                {'metabs': 0, '16s': 10, 'scfa': 0, 'toxin':0}, 
                var_perc = {'metabs': 15, '16s': 5, 'scfa': 0, 'toxin':0})


/Users/jendawk/Dropbox (MIT)/C Diff Recurrence Paper/Analyses/scripts


/Users/jendawk/miniconda3/envs/CodeBase/lib/python3.7/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


# Define functions for doing over representation analysis

In [18]:
import math
def combination(n,k):
    return math.factorial(n)/(math.factorial(k)*math.factorial(n-k))

def ora(x, X):
    N = len(X)
    n = len(x)
#     print('n=' + str(n))
    num_hits = {}
    num_hits_expected = {}
    prob = {}
    for group in np.unique(x):
        k = np.sum(np.array(x)==group)
        K = np.sum(np.array(X)==group)
        if K < 4 or k < 3 or group == 'None' or group == 'Partially Characterized Molecules':
#             print(group + ': K='+str(K) + ', k=' + str(k))
            continue
        num_hits[group] = k
        num_hits_expected[group] = (K/N)*n
        prob[group] = (combination(K,k)*combination(N-K, n-k))/combination(N,n)
    if len(prob)==0:
        return None

    gps, pvals = zip(*prob.items())
    reject, corrected, a1, a2 = multipletests(list(pvals), alpha=.05, method='fdr_bh')
    fdr = {gp:corr for gp, corr in zip(gps, corrected)}
    out_df = pd.DataFrame({'# Hits':num_hits, '# Expected Hits': num_hits_expected, 'p-value': prob, 'FDR': fdr})
    return out_df.sort_values('p-value', ascending = True)

# Enrichment analysis for metabolites

In [63]:
# Set path to where univariate analysis results of metabolites is stored
path = path_to_save + '/univariate_analysis/metabs/'
save_dict = {}
keys = []
vals = []
for file in os.listdir(path):
    if '.csv' not in file:
        continue
    if 'ranksum' in file:
        continue
        
    df = pd.read_csv(path + file, index_col = 0)
    mets = df.index.values[df['BH corrected']<= 0.05]
    if 'rrer' in file:
        weeks = file.split('metabs')[1].split('_')[:-1]
        outcome = file.split('_')[-1].split('.')[0]
        a_type = 'Intra-group, ' + outcome + ', week ' + str(weeks[0]) + ' vs ' + str(weeks[1])
    else:
        a_type = 'Inter-group'

    for pathway in ['SUPER_PATHWAY','SUB_PATHWAY']:
        K = dl.col_mat_mets[pathway].loc[df.index.values]
        k = dl.col_mat_mets[pathway].loc[mets]
#         print(file)
#         print(pathway)
#         print(ora(k,K))
        val = ora(k,K)
        if val is not None:
            keys.append((a_type, pathway))
            vals.append(val.append(pd.Series(['']*val.shape[1], name = '_')))
#         save_dict['Metabolites'][(a_type, pathway)] = ora(k,K)
#         print('')
out_df_mets = pd.concat(vals, axis = 0, keys = keys)

# Enrichment analysis for ASVs

In [64]:
path = path_to_save + '/univariate_analysis/16s/'
path_taxa = 'inputs/'
taxa = pd.read_csv(path_taxa + 'dada2-taxonomy-rdp.csv', index_col = 0)
save_dict['ASVs'] = {}
keys = []
vals = []
for file in os.listdir(path):
    if '.csv' not in file:
        continue
        
    df = pd.read_csv(path + file, index_col = 0)
    FAM = taxa[df.index.values].loc['Family'].dropna()
    mets = df.index.values[df['padj']<= 0.05]
    fam = taxa[mets].loc['Family'].dropna()
    val = ora(fam, FAM)
    
    if 'rrer' in file:
        weeks = file.split('rrer')[1].split('.')[0].split('_')
        outcome = file.split('_')[1].split('s')[1].split(weeks[0])[0]
        a_type = 'Intra-group, ' + outcome + ', week ' + str(weeks[0]) + ' vs ' + str(weeks[1])
    else:
        a_type = 'Inter-group'
        
    if val is not None:
        keys.append((a_type, 'Family'))
        vals.append(val.append(pd.Series(['']*val.shape[1], name = '_')))
out_df_ASVs = pd.concat(vals, axis = 0, keys = keys)

In [65]:
path_save = save_path + '/Supplemental Tables/'
fname = 'Table 3. Enrichment Analysis.xlsx'
with pd.ExcelWriter(path_save + fname, mode = 'w') as writer:
    out_df_ASVs.to_excel(writer, sheet_name = 'ASVs')
    
with pd.ExcelWriter(path_save + fname, mode = 'a') as writer:
    out_df_mets.to_excel(writer, sheet_name = 'Metabolites')